# Preprocessing code

## Data Gathering

In [17]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import requests

# To unzip the edf_dataset
import zipfile
import os

# EDFlib and Data Preprocesing module
from mne.preprocessing import ICA, create_eog_epochs
import mne
from pyedflib import highlevel
import pyedflib as plib

In [18]:
import requests


def download_file(url, save_path):

    # Check if the file already exists
    if os.path.exists(save_path):
        print(f"File already exists at '{save_path}'. Skipping download.")
        return  # Exit the function if the file exists

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in binary write mode and save the content
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded successfully and saved to '{save_path}'")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")


# Specify the URL and the path where you want to save the file
url = 'https://figshare.com/ndownloader/articles/4244171/versions/2'
# Change this to your desired path
save_path = '../edf_dataset.zip'

# Call the function to download the file
download_file(url, save_path)

File downloaded successfully and saved to './edf_dataset.zip'


In [19]:
def unzip_file(zip_file_path, extract_to_folder):

    # Check if the directory exist
    
    if os.path.exists(extract_to_folder):
        print(f"Directory '{extract_to_folder} already exists")
        return # Exit the function if the directory
    
    # Create the directory if it doesn't exist
    os.makedirs(extract_to_folder, exist_ok=True)

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents into the specified folder
        zip_ref.extractall(extract_to_folder)


# Specify the path to the zip file and the extraction folder
zip_file_path = '../edf_dataset.zip'
# Change this if needed
extract_to_folder = '../edf_dataset_2'

# Call the function to unzip
unzip_file(zip_file_path, extract_to_folder)


In [4]:
import os
import re

edf_directory = "./edf_dataset_2"

for filename in os.listdir(edf_directory):
    new_filename = filename

    # Handle spaces in filename
    if ' ' in new_filename:
        new_filename = re.sub(r'\s+', '_', new_filename)

    # Handle subject numbers (S1 -> S01)
    new_filename = re.sub(r'S(\d)(?!\d)', r'S0\1', new_filename)

    # Handle filenames containing '6931959'
    if '6931959' in new_filename:
        new_filename = new_filename.replace('6931959_', '')

    # Handle filenames containing '6921143'
    if '6921143' in new_filename:
        new_filename = new_filename.replace('6921143_', '')

    # Zero-pad other single-digit numbers in filenames
    new_filename = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', new_filename)

    # Only rename if the filename has changed
    if new_filename != filename:
        old_file = os.path.join(edf_directory, filename)
        new_file = os.path.join(edf_directory, new_filename)

        os.rename(old_file, new_file)
        print(f'Renamed: "{filename}" to "{new_filename}"')

print("Renaming complete.")

Renamed: "H S7 TASK.edf" to "H_S07_TASK.edf"
Renamed: "H S6 TASK.edf" to "H_S06_TASK.edf"
Renamed: "H S16 EO.edf" to "H_S16_EO.edf"
Renamed: "MDD S17 EC.edf" to "MDD_S17_EC.edf"
Renamed: "H S13 EC.edf" to "H_S13_EC.edf"
Renamed: "MDD S12 EO.edf" to "MDD_S12_EO.edf"
Renamed: "H S5 EC.edf" to "H_S05_EC.edf"
Renamed: "H S25 EC.edf" to "H_S25_EC.edf"
Renamed: "MDD S24 EO.edf" to "MDD_S24_EO.edf"
Renamed: "MDD S24  EC.edf" to "MDD_S24_EC.edf"
Renamed: "MDD S4 EO.edf" to "MDD_S04_EO.edf"
Renamed: "MDD S7  EC.edf" to "MDD_S07_EC.edf"
Renamed: "MDD S33 EC.edf" to "MDD_S33_EC.edf"
Renamed: "MDD S1 EC.edf" to "MDD_S01_EC.edf"
Renamed: "MDD S13 TASK.edf" to "MDD_S13_TASK.edf"
Renamed: "MDD S12 TASK.edf" to "MDD_S12_TASK.edf"
Renamed: "H S29 EC.edf" to "H_S29_EC.edf"
Renamed: "H S9 EC.edf" to "H_S09_EC.edf"
Renamed: "H S20 EO.edf" to "H_S20_EO.edf"
Renamed: "MDD S28 EO.edf" to "MDD_S28_EO.edf"
Renamed: "MDD S21 EC.edf" to "MDD_S21_EC.edf"
Renamed: "MDD S27 TASK.edf" to "MDD_S27_TASK.edf"
Renamed: 

In [6]:
all_edf_files = os.listdir(edf_directory)
ec_file_path = [i for i in all_edf_files if i.endswith('EC.edf')]
eo_file_path = [i for i in all_edf_files if i.endswith('EO.edf')]
task_file_path = [i for i in all_edf_files if i.endswith('TASK.edf')]

print(len(all_edf_files), len(ec_file_path),
      len(eo_file_path), len(task_file_path))

181 58 62 61


## Preprocessing Functions

In [5]:
# Function to rename channels and drop specified channels based on conditions
def process_channels(raw_data):
    """
    Process and standardize EEG channels to keep only the 17 most common channels.
    """
    print(f"Initial channels: {raw_data.ch_names}")

    # Initialize a list to hold channels to drop
    channels_to_drop = []

    # Create mapping for channel renaming
    rename_map = {}
    for name in raw_data.ch_names:
        if any(x in name for x in ['23A-23R', '24A-24R', 'A2-A1']):
            channels_to_drop.append(name)
        else:
            new_name = name.replace('EEG ', '').replace('-LE', '')
            rename_map[name] = new_name

    # Drop unwanted channels
    if channels_to_drop:
        print(f"Dropping channels: {channels_to_drop}")
        raw_data.drop_channels(channels_to_drop)

    # Rename remaining channels
    raw_data.rename_channels(rename_map)

    print(f"Final channels: {raw_data.ch_names}")

    # Define the 17 most common channels
    expected_channels = [
        'Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'Fp2',
        'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Cz', 'Pz'
    ]

    # Keep only the expected channels
    channels_to_keep = set(expected_channels)
    channels_to_drop = [
        ch for ch in raw_data.ch_names if ch not in channels_to_keep]

    if channels_to_drop:
        print(
            f"Dropping channels to keep only the expected 17 channels: {channels_to_drop}")
        raw_data.drop_channels(channels_to_drop)

    # Verify we have the expected number of channels (should be 17)
    if len(raw_data.ch_names) != len(expected_channels):
        print(
            f"Warning: Expected {len(expected_channels)} channels, got {len(raw_data.ch_names)}")
        print(f"Missing: {set(expected_channels) - set(raw_data.ch_names)}")

    return raw_data

In [7]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    return data

In [9]:
# Bandpass filtering
def bandpass_filter(data, l_freq, h_freq, notch_freq=None):
    filtered_data = data.copy()

    # Apply bandpass first
    filtered_data.filter(l_freq=l_freq, h_freq=h_freq,
                         method='fir', phase='zero')

    # If using notch, apply with wider bandwidth
    if notch_freq is not None:
        filtered_data.notch_filter(freqs=notch_freq, notch_widths=2.0)

    return filtered_data

In [8]:
ica_channels = ['Fp1', 'Fp2']

In [10]:
def preprocess_ICA(epochs, n_components):
    """
    Apply Independent Component Analysis (ICA) to the epochs data.
    
    Parameters:
    -----------
    epochs : mne.Epochs
        The epochs data to process.
    n_components : int
        The number of components to extract.
    
    Returns:
    --------
    ica : ICA
        The fitted ICA object.
    """
    print(
        f"Preprocessing ICA for {len(epochs)} epochs...")  # Print the number of epochs being processed

    ica = ICA(n_components=n_components, random_state=97, max_iter=800)
    # Use the epochs directly
    ica.fit(epochs.copy().pick_channels(ica_channels))
    return ica

In [11]:
def create_epochs(processed_data, duration=5.0, overlap=1.0):
    """
    Create epochs from continuous EEG data and format for CNN input
    
    Parameters:
    -----------
    processed_data : mne.io.Raw
        The raw EEG data
    duration : float
        Duration of each epoch in seconds
    overlap : float
        Overlap between epochs in seconds
    
    Returns:
    --------
    epochs_array : numpy.ndarray
        The epoched data formatted for CNN (samples, channels, timepoints, 1)
    """

    # Create epochs
    epochs = mne.make_fixed_length_epochs(
        processed_data,
        duration=duration,
        overlap=overlap,
        preload=True
    )

    # Drop bad epochs
    epochs.drop_bad()

    # Get data and reshape for CNN
    # Shape will be (n_epochs, n_channels, n_timepoints)
    data = epochs.get_data()

    # Add channel dimension for CNN: (n_epochs, n_channels, n_timepoints, 1)
    data = data[..., np.newaxis]

    return data

In [12]:
# Directory containing the EDF files
edf_directory = "./edf_dataset_2"  # Adjust this path to your dataset location

# Initialize lists
processed_raw_data = []
class_counts = {'Healthy': 0, 'MDD': 0}

# Read all EDF files
for filename in os.listdir(edf_directory):
    if filename.endswith('.edf'):
        file_path = os.path.join(edf_directory, filename)
        try:
            # Read the raw data
            raw_data = read_data(file_path)

            if raw_data is not None:
                processed_raw_data.append(raw_data)
                print(f"Successfully loaded: {filename}")
            else:
                print(f"Failed to load: {filename}")

        except Exception as e:
            print(f"Error loading {filename}: {str(e)}")
            continue

print(f"\nTotal files loaded: {len(processed_raw_data)}")

Extracting EDF parameters from /Users/hansandreanto/Development/capstone-project/core/edf_dataset_2/H_S08_EO.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77311  =      0.000 ...   301.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Successfully loaded: H_S08_EO.edf
Extracting EDF parameters from /Users/hansandreanto/Development/capstone-project/core/edf_dataset_2/H_S01_EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 76799  =      0.000 ...   299.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Successfully loaded: H_S01_EC.edf
Extracting EDF parameters from /Users/hansandreanto/Development/capstone-project/core/edf_dataset_2/MDD_S09_EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

In [14]:
# The actual preprocessing data
def preprocess_eeg(raw_data, l_freq=0.5, h_freq=60.0, notch_freq=50.0, n_components=2, epoch_duration=5.0, epoch_overlap=1.0):
    """
    Complete EEG preprocessing pipeline: filter -> bad channel removal -> epoching -> ICA -> baseline correction
    """
    try:
        # print(f"\nProcessing file: {filename}")

        # Make a copy of the raw data to prevent modifications to original
        processed_raw = raw_data.copy()

        # 1. Bandpass filtering
        print("1. Applying bandpass filter...")
        try:
            bandpass_filter(processed_raw, l_freq, h_freq, notch_freq)
            print("Bandpass filtering completed")
        except Exception as e:
            print(f"Error during bandpass filtering: {str(e)}")
            return None

        # 2. Bad channel removal
        print("2. Removing bad channels...")
        try:
            processed_raw = process_channels(raw_data=processed_raw)
            print("Bad channels removed")
        except Exception as e:
            print(f"Error during bad channel removal: {str(e)}")
            return None

        # 3. Epoching
        print("3. Creating epochs...")
        try:
            epochs = mne.make_fixed_length_epochs(
                processed_raw,
                duration=epoch_duration,
                overlap=epoch_overlap,
                preload=True
            )

            # Drop bad epochs
            epochs.drop_bad()

            # Get data and reshape for CNN
            data = epochs.get_data()
            data = data[..., np.newaxis]  # Add channel dimension for CNN

            print(f"Epoching completed. Final data shape: {data.shape}")
        except Exception as e:
            print(f"Error during epoching: {str(e)}")
            return None

        # 4. ICA
        print("4. Applying ICA...")
        try:
            # Pass the epochs object
            ica = preprocess_ICA(epochs, n_components)
            ica.apply(epochs)  # Apply ICA to the epochs
            print("ICA completed")
        except Exception as e:
            print(f"Error during ICA: {str(e)}")
            return None

        # 5. Baseline correction
        print("5. Applying baseline correction...")
        try:
            # Apply baseline correction over the entire epoch
            epochs.apply_baseline((None, None))
            print("Baseline correction completed")
        except Exception as e:
            print(f"Error during baseline correction: {str(e)}")
            return None

        return data  # Return the processed data

    except Exception as e:
        print(f"General preprocessing error: {str(e)}")
        return None

In [15]:

# Now process each raw data file
X_data = []
y_labels = []
class_counts = {'Healthy': 0, 'MDD': 0}

print("\nStarting preprocessing pipeline...")
for raw_data in processed_raw_data:
    # Get the filename from the raw data
    filename = os.path.basename(raw_data.filenames[0])
    print(f"\n{'='*50}")
    print(f"Processing: {filename}")
    print(f"Initial data info:")
    print(f"Channels: {raw_data.ch_names}")
    print(f"Sample rate: {raw_data.info['sfreq']} Hz")
    print(f"Duration: {raw_data.n_times / raw_data.info['sfreq']:.2f} seconds")

    # Apply complete preprocessing pipeline
    try:
        # Apply complete preprocessing pipeline
        processed_data = preprocess_eeg(
            raw_data,

            l_freq=0.5,
            h_freq=50.0,
            n_components=2,
            notch_freq=50.0,
            epoch_duration=5.0,
            epoch_overlap=1.0
        )

        if processed_data is not None:
            print(f"Processed data shape: {processed_data.shape}")

            # Create label (1 for MDD, 0 for Healthy)
            label = 1 if filename.startswith('MDD') else 0

            # Update counts with number of epochs
            if label == 1:
                class_counts['MDD'] += processed_data.shape[0]
            else:
                class_counts['Healthy'] += processed_data.shape[0]

            # Append to lists
            X_data.append(processed_data)
            y_labels.extend([label] * processed_data.shape[0])
            print(f"Successfully processed {filename}")
        else:
            print(f"Failed to process {filename}")

    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        continue

    print(f"{'='*50}\n")


# Print final summary
print("\nProcessing Summary:")
print(f"Total files processed: {len(processed_raw_data)}")
print(f"Total epochs: {len(y_labels)}")
print(f"Successfully processed files: {len(X_data)}")
print(f"Failed files: {len(processed_raw_data) - len(X_data)}")
print("\nClass distribution:")
print(f"MDD epochs: {class_counts['MDD']}")
print(f"Healthy epochs: {class_counts['Healthy']}")


Starting preprocessing pipeline...

Processing: H_S08_EO.edf
Initial data info:
Channels: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1', 'EEG 23A-23R', 'EEG 24A-24R']
Sample rate: 256.0 Hz
Duration: 302.00 seconds
1. Applying bandpass filter...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA to data using 2 channels (please be patient, this may take a while)
Selecting by number: 2 components
Fitting ICA took 0.8s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 2 PCA components
ICA completed
5. Applying baseline correction...
Applying baseline correction (mode: mean)
Baseline correction completed
Processed data shape: (75, 17, 1280, 1)
Successfully processed H_S08_EO.edf


Processing: H_S01_EC.edf
Initial data info:
Channels: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1', 'EEG 23A-23R', 'EEG 24A-24R']
Sample rate: 256.0 Hz
Duration: 300.00 seconds
1. Applying bandpass filter...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Fitting ICA took 0.3s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 2 PCA components
ICA completed
5. Applying baseline correction...
Applying baseline correction (mode: mean)
Baseline correction completed
Processed data shape: (74, 17, 1280, 1)
Successfully processed H_S01_EC.edf


Processing: MDD_S09_EC.edf
Initial data info:
Channels: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1']
Sample rate: 256.0 Hz
Duration: 300.00 seconds
1. Applying bandpass filter...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (fi

...

Fitting ICA took 0.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 2 PCA components
ICA completed
5. Applying baseline correction...
Applying baseline correction (mode: mean)
Baseline correction completed
Processed data shape: (75, 17, 1280, 1)
Successfully processed H_S29_EO.edf


Processing: MDD_S21_EO.edf
Initial data info:
Channels: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1']
Sample rate: 256.0 Hz
Duration: 300.00 seconds
1. Applying bandpass filter...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (fi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Fitting ICA took 0.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 2 PCA components
ICA completed
5. Applying baseline correction...
Applying baseline correction (mode: mean)
Baseline correction completed
Processed data shape: (74, 17, 1280, 1)
Successfully processed MDD_S21_EO.edf


Processing: MDD_S28_EC.edf
Initial data info:
Channels: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1']
Sample rate: 256.0 Hz
Duration: 301.00 seconds
1. Applying bandpass filter...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 2 PCA components
ICA completed
5. Applying baseline correction...
Applying baseline correction (mode: mean)
Baseline correction completed
Processed data shape: (75, 17, 1280, 1)
Successfully processed MDD_S28_EC.edf


Processing Summary:
Total files processed: 181
Total epochs: 18387
Successfully processed files: 181
Failed files: 0

Class distribution:
MDD epochs: 9756
Healthy epochs: 8631


In [16]:
# Check if we have any processed data
if len(X_data) > 0:
    try:
        # Print shapes before concatenation
        print("\nArray shapes before concatenation:")
        for i, arr in enumerate(X_data):
            print(f"Array {i}: shape {arr.shape}")

        # Concatenate data
        X = np.concatenate(X_data, axis=0)
        y = np.array(y_labels)

        # Print final information
        print("\nFinal Dataset Information:")
        print(f"Total samples: {len(X)}")
        print(f"Healthy samples: {class_counts['Healthy']}")
        print(f"MDD samples: {class_counts['MDD']}")
        print(f"Input shape: {X.shape}")
        print(f"Labels shape: {y.shape}")

        # Print channel information
        print(f"\nChannels used: {raw_data.ch_names}")

        # Print class balance
        total_epochs = class_counts['MDD'] + class_counts['Healthy']
        print("\nClass balance:")
        print(f"MDD: {(class_counts['MDD']/total_epochs)*100:.2f}%")
        print(f"Healthy: {(class_counts['Healthy']/total_epochs)*100:.2f}%")

    except Exception as e:
        print(f"\nError during final processing: {str(e)}")
        print("Checking individual arrays for inconsistencies...")

        # Find arrays with different shapes
        base_shape = X_data[0].shape[1:]
        for i, arr in enumerate(X_data):
            if arr.shape[1:] != base_shape:
                print(
                    f"Mismatch at index {i}: expected {base_shape}, got {arr.shape[1:]}")
else:
    print("\nNo data was successfully processed!")


Array shapes before concatenation:
Array 0: shape (75, 17, 1280, 1)
Array 1: shape (74, 17, 1280, 1)
Array 2: shape (74, 17, 1280, 1)
Array 3: shape (74, 17, 1280, 1)
Array 4: shape (74, 17, 1280, 1)
Array 5: shape (74, 17, 1280, 1)
Array 6: shape (47, 17, 1280, 1)
Array 7: shape (73, 17, 1280, 1)
Array 8: shape (159, 17, 1280, 1)
Array 9: shape (152, 17, 1280, 1)
Array 10: shape (75, 17, 1280, 1)
Array 11: shape (75, 17, 1280, 1)
Array 12: shape (74, 17, 1280, 1)
Array 13: shape (74, 17, 1280, 1)
Array 14: shape (151, 17, 1280, 1)
Array 15: shape (79, 17, 1280, 1)
Array 16: shape (158, 17, 1280, 1)
Array 17: shape (75, 17, 1280, 1)
Array 18: shape (158, 17, 1280, 1)
Array 19: shape (74, 17, 1280, 1)
Array 20: shape (75, 17, 1280, 1)
Array 21: shape (162, 17, 1280, 1)
Array 22: shape (157, 17, 1280, 1)
Array 23: shape (158, 17, 1280, 1)
Array 24: shape (157, 17, 1280, 1)
Array 25: shape (151, 17, 1280, 1)
Array 26: shape (151, 17, 1280, 1)
Array 27: shape (151, 17, 1280, 1)
Array 28: 